### **파일 로드**

In [ ]:
from google.colab import files

uploaded = files.upload()  # 파일 선택 창이 뜸

Saving 전체상담기록_2025-05-10_21-30.txt to 전체상담기록_2025-05-10_21-30.txt
Saving 전체상담기록_2025-05-10_22-58.txt to 전체상담기록_2025-05-10_22-58.txt
Saving 전체상담기록_2025-05-13_02-13.txt to 전체상담기록_2025-05-13_02-13.txt
Saving 전체상담기록_2025-05-19_15-12.txt to 전체상담기록_2025-05-19_15-12.txt
Saving 전체상담기록_2025-05-28_23-06.txt to 전체상담기록_2025-05-28_23-06.txt
Saving 전체상담기록_2025-05-28_23-51.txt to 전체상담기록_2025-05-28_23-51.txt
Saving 전체상담기록_2025-06-12_13-59.txt to 전체상담기록_2025-06-12_13-59.txt
Saving 전체상담기록_2025-07-03_02-44.txt to 전체상담기록_2025-07-03_02-44.txt
Saving 전체상담기록_2025-07-03_02-59.txt to 전체상담기록_2025-07-03_02-59.txt


<br>

### **태깅 실행**

In [ ]:
from openai import OpenAI
import os

# OpenAI 클라이언트 생성
client = OpenAI(api_key="OPENAI_API_KEY")
# 보고서용 → 상담사용 태그 매핑
REPORT_TO_COUNSEL_TAG_MAP = {
    "성학대": "아동 성학대",
    "신체학대": "아동 신체학대",
    "정서불안": "정서•행동 문제",
    "자해/자살 생각": "정서•행동 문제",
    "외모/신체 고민": "정서•행동 문제",
    "학교폭력": "학교폭력",
    "집단활동/교우관계 불안": "또래•사회관계 문제",
    "가정학대": "가정폭력/양육방임",
    "양육방임": "가정폭력/양육방임",
    "부모의 기대/훈육 스트레스": "부모•가족관계 문제",
    "형제자매 갈등": "부모•가족관계 문제",
    "부모와의 갈등": "부모•가족관계 문제",
    "이혼/별거": "부모•가족관계 문제",
    "학업스트레스": "학업•진로 스트레스",
    "가정형편 어려움": "경제적•가정형편 문제",
    "해당없음": "해당없음"
}

# 태깅 주제 리스트()
TAG_TOPICS = [
    "성학대", "신체학대", "정서불안", "학교폭력", "집단활동/교우관계 불안", "가정학대",
    "자해/자살 생각", "학업스트레스", "부모의 기대/훈육 스트레스", "형제자매 갈등","부모와의 갈등",
    "이혼/별거", "양육방임", "외모/신체 고민", "가정형편 어려움","해당없음"
]

# 태깅 함수 정의
def tag_conversation(conversation_text):
    system_prompt = (
        "너는 아동·청소년 심리상담 데이터를 분석해서, 아래 주제 목록 중 대화 내용에 **명확하게 해당되는 항목만 정확히 태깅하는 모델**이야.\n\n"
        "🔖 태깅 가능한 주제 목록:\n"
        "성학대, 신체학대, 정서불안, 학교폭력, 집단활동/교우관계 불안, 가정학대, 자해/자살 생각, 학업스트레스,\n"
        "부모의 기대/훈육 스트레스, 형제자매 갈등, 부모와의 갈등, 이혼/별거, 양육방임, 외모/신체 고민, 가정형편 어려움, 해당없음\n\n"

        "태깅 규칙:\n"
        "1. 대화 내용에 **직접적으로 드러난 내용만 태깅**해. 추측이나 연상은 절대 금지야. **대화에 명확하게 드러나지 않은 내용은 절대 태그하지 마.**\n"
        "2. **해당되는 주제가 여러 개면 모두 태깅**해. 단, 중복 태깅은 금지.\n"
        "3. '해당없음'은 오직 **태깅된 주제가 하나도 없을 때만** 태깅해. 다른 태그가 있으면 절대 포함하지 마.\n"
        "4. **사소한 말다툼, 장난, 감정 기복**은 태깅하지 마.\n"
        "5. 태그는 반드시 **주제 목록 중에서만** 골라. 목록에 없는 주제는 절대 포함하지 마.\n\n"

        "📌 주제별 판단 기준:\n"
        "- 성학대: 성인이 아동에게 성적인 의도 또는 불쾌감을 유발하는 신체 접촉을 하거나 시도한 경우. 아동이 불쾌함이나 거부 의사를 명확히 표현해야 함.\n"
        "- 신체학대: 성인이 아동에게 명백한 물리적 폭력을 가한 경우 (예: 때림, 밀침, 물건 던지기 등). 단순 신체 접촉은 아님.\n"
        "- 정서불안: 아동이 불안, 슬픔, 무기력, 불면, 감정 통제 어려움 등을 반복적으로 드러낸 경우. 단순 기분 변화는 해당되지 않음.\n"
        "- 학교폭력: 또래의 일방적 괴롭힘, 따돌림이 반복되거나 아동이 고통을 호소하는 경우. 장난은 해당하지 않음.\n"
        "- 집단활동/교우관계 불안: 교우관계에서의 불안, 소외감, 모둠 활동 어려움 등이 명확히 드러난 경우.\n"
        "- 가정학대: 부모 또는 보호자로부터 신체적 또는 정서적 학대가 명확히 드러난 경우.\n"
        "- 자해/자살 생각: 아동이 스스로를 다치게 하려 하거나 죽고 싶다는 표현이 명확히 드러난 경우.\n"
        "- 학업스트레스: 공부, 성적, 시험에 대한 과도한 압박감이나 스트레스가 명확한 경우.\n"
        "- 부모의 기대/훈육 스트레스: 부모의 과도한 기대나 엄격한 훈육으로 인한 스트레스가 드러난 경우.\n"
        "- 형제자매 갈등: 형제자매 사이의 갈등, 비교, 싸움 등이 명확한 경우.\n"
        "- 부모와의 갈등: 부모와의 대화, 의견 충돌, 갈등이 드러난 경우.\n"
        "- 이혼/별거: 부모가 이혼했거나 별거 중이라는 사실이 대화에서 명확히 언급된 경우.\n"
        "- 양육방임: 보호자가 아동을 방치하거나 돌보지 않는 상황이 명확한 경우.\n"
        "- 외모/신체 고민: 키, 몸무게, 얼굴 등 외모에 관한 고민이나 스트레스가 드러난 경우.\n"
        "- 가정형편 어려움: 집안의 경제적 어려움이 아동 대화에서 명확히 드러난 경우.\n"
        "- 해당없음: 위 주제에 해당하지 않는 경우에만 태깅.\n\n"



        "📌 주의사항:\n"
        "- 추측, 억측, 모호한 내용은 절대 태깅하지 마.\n"
        "- 같은 주제를 중복 태깅하지 마.\n"
        "- 태그 목록에 없는 주제는 절대 태깅하지 마.\n"
        "- 장난, 일시적 감정 변화 등은 태깅하지 마.\n\n"

        "🧾 출력 형식:\n"
        "태깅 결과: {\"tags\": [\"주제1\", \"주제2\", ...]}\n\n"
        "- 주제1: 태그가 선택된 이유\n"
        "- 주제2: 태그가 선택된 이유\n\n"
        "반드시 위와 같은 형식을 유지해서 출력하고, 불필요한 설명은 생략하세요. 항목별로 줄바꿈과 '- ' 기호를 사용해서 나열하세요.\n"
        "태그가 하나여도 동일한 형식으로 출력하세요.\n"
        "중복 태그 없이 출력하고, '해당없음'은 다른 태그가 없을 때만 사용하세요.\n"

    )

    response = client.chat.completions.create(
        model="gpt-4o",
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": conversation_text}
        ]
    )

    # 응답 파싱
    result_text = response.choices[0].message.content

    # 보고서용 태그 추출
    try:
        report_tags_line = result_text.split('\n')[0]
        report_tags = eval(report_tags_line.replace("태깅 결과:", "").strip())["tags"]
    except Exception as e:
        print("태그 추출 실패:", e)
        return None

    # 상담사용 태그 매핑
    counsel_tags_set = set()
    for tag in report_tags:
        if tag in REPORT_TO_COUNSEL_TAG_MAP:
            counsel_tags_set.add(REPORT_TO_COUNSEL_TAG_MAP[tag])

    # 중복 제거 후 정렬
    counsel_tags = sorted(counsel_tags_set)

    # 결과 반환 (프론트에서 위치 다르게 보여줄 수 있도록 분리)
    return {
        "report_tags": report_tags,
        "counsel_tags": counsel_tags,
        "reasoning_text": result_text
    }

</br>

#### **전체상담기록 파일에 태깅한 결과**

In [ ]:
# 수정한 출력
import os

# 파일 경로 설정
file_path = "전체상담기록_2025-05-10_21-30.txt"

# 파일에서 대화 내용 읽고 태깅 실행
if os.path.exists(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation_data = file.read()
        result = tag_conversation(conversation_data)

        # 결과 출력 (가독성 있는 출력용)
        print("보고서용 태그:", result["report_tags"])
        print("상담사용 태그:", result["counsel_tags"])

        print("\n태깅 사유:")
        reasoning_lines = result["reasoning_text"].split('\n')
        for line in reasoning_lines[1:]:  # 첫 줄(태깅 결과:)은 생략
          print(line)

else:
    print(f"❌ 파일을 찾을 수 없습니다: {file_path}")

보고서용 태그: ['신체학대', '가정학대', '정서불안', '부모와의 갈등']
상담사용 태그: ['가정폭력/양육방임', '부모•가족관계 문제', '아동 신체학대', '정서•행동 문제']

태깅 사유:

- 신체학대: 아동이 아빠가 술을 마시고 컵을 던져 팔을 다쳤다고 명확히 언급함.
- 가정학대: 아빠가 화를 내며 물건을 던지는 상황이 반복적으로 언급됨.
- 정서불안: 부모님의 싸움으로 인해 불안하고 눈물이 난다고 표현함.
- 부모와의 갈등: 아빠와의 대화가 혼날 때만 이루어지고, 아빠가 화를 내며 물건을 던지는 상황에서 무서움을 느낀다고 언급함.


In [ ]:
import os
import json

# 파일 경로 설정
file_path = "전체상담기록_2025-05-10_21-30.txt"

# 파일에서 대화 내용 읽고 태깅 실행
if os.path.exists(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation_data = file.read()
        result = tag_conversation(conversation_data)

        # 보고서 태그 + 사유
        report_data = {
            "report_tags": result["report_tags"],
            "tag_reasons": "\n".join(result["reasoning_text"].split('\n')[1:])  # 첫 줄 제거
        }

        # 상담사용 태그
        counsel_data = {
            "counsel_tags": result["counsel_tags"]
        }

        # 결과 확인 출력
        print("\n보고서용 태그:", report_data["report_tags"])
        print("\n태깅 근거:", report_data["tag_reasons"])
        print("\n상담사용 태그:", counsel_data["counsel_tags"])

        # 결과를 각각 저장하거나 프론트로 넘기기 쉽게 분리
        with open("보고서용_태그결과.json", "w", encoding="utf-8") as f:
            json.dump(report_data, f, ensure_ascii=False, indent=2)

        with open("상담사용_태그결과.json", "w", encoding="utf-8") as f:
            json.dump(counsel_data, f, ensure_ascii=False, indent=2)

else:
    print(f"❌ 파일을 찾을 수 없습니다: {file_path}")



보고서용 태그: ['신체학대', '가정학대', '정서불안', '부모와의 갈등']

태깅 근거: 
- 신체학대: 아동이 아빠가 술을 마시고 컵을 던져 팔을 다쳤다고 명확히 언급함.
- 가정학대: 아빠가 화를 내며 물건을 던지는 상황이 반복적으로 언급됨.
- 정서불안: 부모님의 싸움으로 인해 불안하고 눈물이 난다고 표현함.
- 부모와의 갈등: 아빠와의 관계에서 엄격하고 무서운 존재로 인식하고 있으며, 혼날 때만 대화한다고 언급함.

상담사용 태그: ['가정폭력/양육방임', '부모•가족관계 문제', '아동 신체학대', '정서•행동 문제']


  <br>

In [ ]:
# 파일 경로 설정
file_path = "전체상담기록_2025-05-10_22-58.txt"

# 파일에서 대화 내용 읽고 태깅 실행
if os.path.exists(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation_data = file.read()
        result = tag_conversation(conversation_data)

        # 보고서 태그 + 사유
        report_data = {
            "report_tags": result["report_tags"],
            "tag_reasons": "\n".join(result["reasoning_text"].split('\n')[1:])  # 첫 줄 제거
        }

        # 상담사용 태그
        counsel_data = {
            "counsel_tags": result["counsel_tags"]
        }

        # 결과 확인 출력
        print("\n보고서용 태그:", report_data["report_tags"])
        print("\n태깅 근거:", report_data["tag_reasons"])
        print("\n상담사용 태그:", counsel_data["counsel_tags"])

        # 결과를 각각 저장하거나 프론트로 넘기기 쉽게 분리
        with open("보고서용_태그결과.json", "w", encoding="utf-8") as f:
            json.dump(report_data, f, ensure_ascii=False, indent=2)

        with open("상담사용_태그결과.json", "w", encoding="utf-8") as f:
            json.dump(counsel_data, f, ensure_ascii=False, indent=2)

else:
    print(f"❌ 파일을 찾을 수 없습니다: {file_path}")



보고서용 태그: ['학교폭력', '집단활동/교우관계 불안']

태깅 근거: 
- 학교폭력: 아동이 같은 반 친구들로부터 단체로 놀림을 당하고, 그로 인해 자존심이 상하고 부끄러워서 눈물이 났다고 표현함.
- 집단활동/교우관계 불안: 아동이 친구가 없고, 친구를 사귀지 못할까봐 걱정된다고 표현함. 친구들과의 관계에서 소외감을 느끼고 있음.

상담사용 태그: ['또래•사회관계 문제', '학교폭력']


#  

In [ ]:
# 파일 경로 설정
file_path = "전체상담기록_2025-05-13_02-13.txt"

# 파일에서 대화 내용 읽고 태깅 실행
if os.path.exists(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation_data = file.read()
        result = tag_conversation(conversation_data)

        # 보고서 태그 + 사유
        report_data = {
            "report_tags": result["report_tags"],
            "tag_reasons": "\n".join(result["reasoning_text"].split('\n')[1:])  # 첫 줄 제거
        }

        # 상담사용 태그
        counsel_data = {
            "counsel_tags": result["counsel_tags"]
        }

        # 결과 확인 출력
        print("\n보고서용 태그:", report_data["report_tags"])
        print("\n태깅 근거:", report_data["tag_reasons"])
        print("\n상담사용 태그:", counsel_data["counsel_tags"])

        # 결과를 각각 저장하거나 프론트로 넘기기 쉽게 분리
        with open("보고서용_태그결과.json", "w", encoding="utf-8") as f:
            json.dump(report_data, f, ensure_ascii=False, indent=2)

        with open("상담사용_태그결과.json", "w", encoding="utf-8") as f:
            json.dump(counsel_data, f, ensure_ascii=False, indent=2)

else:
    print(f"❌ 파일을 찾을 수 없습니다: {file_path}")



보고서용 태그: ['성학대', '정서불안', '양육방임']

태깅 근거: 
- 성학대: 학원 선생님이 아동의 몸을 만지려고 하며, 아동이 불쾌함을 명확히 표현함.
- 정서불안: 학원 선생님의 행동으로 인해 아동이 화나고 슬프다고 표현함.
- 양육방임: 부모님이 바빠서 아동이 주로 혼자 집에 있는 상황이 명확히 드러남.

상담사용 태그: ['가정폭력/양육방임', '아동 성학대', '정서•행동 문제']


#  

In [ ]:
# 파일 경로 설정
file_path = "전체상담기록_2025-05-19_15-12.txt"

# 파일에서 대화 내용 읽고 태깅 실행
if os.path.exists(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation_data = file.read()
        result = tag_conversation(conversation_data)

        # 보고서 태그 + 사유
        report_data = {
            "report_tags": result["report_tags"],
            "tag_reasons": "\n".join(result["reasoning_text"].split('\n')[1:])  # 첫 줄 제거
        }

        # 상담사용 태그
        counsel_data = {
            "counsel_tags": result["counsel_tags"]
        }

        # 결과 확인 출력
        print("\n보고서용 태그:", report_data["report_tags"])
        print("\n태깅 근거:", report_data["tag_reasons"])
        print("\n상담사용 태그:", counsel_data["counsel_tags"])

        # 결과를 각각 저장하거나 프론트로 넘기기 쉽게 분리
        with open("보고서용_태그결과.json", "w", encoding="utf-8") as f:
            json.dump(report_data, f, ensure_ascii=False, indent=2)

        with open("상담사용_태그결과.json", "w", encoding="utf-8") as f:
            json.dump(counsel_data, f, ensure_ascii=False, indent=2)

else:
    print(f"❌ 파일을 찾을 수 없습니다: {file_path}")



보고서용 태그: ['양육방임', '정서불안']

태깅 근거: 
- 양육방임: 아동이 저녁마다 배가 고프고, 어머니가 밥을 해놓지 않아 먹지 못한 상황이 반복적으로 발생하며, 아동을 돌봐주는 사람이 없다고 언급함.
- 정서불안: 부모님의 무관심으로 인해 슬픔을 느끼고 있다고 명확히 표현함.

상담사용 태그: ['가정폭력/양육방임', '정서•행동 문제']


#   

In [ ]:
# 파일 경로 설정
file_path = "전체상담기록_2025-05-28_23-06.txt"

# 파일에서 대화 내용 읽고 태깅 실행
if os.path.exists(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation_data = file.read()
        result = tag_conversation(conversation_data)

        # 보고서 태그 + 사유
        report_data = {
            "report_tags": result["report_tags"],
            "tag_reasons": "\n".join(result["reasoning_text"].split('\n')[1:])  # 첫 줄 제거
        }

        # 상담사용 태그
        counsel_data = {
            "counsel_tags": result["counsel_tags"]
        }

        # 결과 확인 출력
        print("\n보고서용 태그:", report_data["report_tags"])
        print("\n태깅 근거:", report_data["tag_reasons"])
        print("\n상담사용 태그:", counsel_data["counsel_tags"])

        # 결과를 각각 저장하거나 프론트로 넘기기 쉽게 분리
        with open("보고서용_태그결과.json", "w", encoding="utf-8") as f:
            json.dump(report_data, f, ensure_ascii=False, indent=2)

        with open("상담사용_태그결과.json", "w", encoding="utf-8") as f:
            json.dump(counsel_data, f, ensure_ascii=False, indent=2)

else:
    print(f"❌ 파일을 찾을 수 없습니다: {file_path}")



보고서용 태그: ['이혼/별거', '정서불안']

태깅 근거: 
- 이혼/별거: 부모님의 이혼으로 인해 아동이 머리가 아프다고 표현하고, 엄마와 따로 살고 있다는 내용이 명확히 드러남.
- 정서불안: 부모님의 이혼으로 인해 불행함을 느끼고, 집안 분위기가 안 좋아서 아무것도 하기 싫다는 등 정서적 불안이 명확히 드러남.

상담사용 태그: ['부모•가족관계 문제', '정서•행동 문제']


#  

In [ ]:
# 파일 경로 설정
file_path = "전체상담기록_2025-05-28_23-51.txt"

# 파일에서 대화 내용 읽고 태깅 실행
if os.path.exists(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation_data = file.read()
        result = tag_conversation(conversation_data)

        # 보고서 태그 + 사유
        report_data = {
            "report_tags": result["report_tags"],
            "tag_reasons": "\n".join(result["reasoning_text"].split('\n')[1:])  # 첫 줄 제거
        }

        # 상담사용 태그
        counsel_data = {
            "counsel_tags": result["counsel_tags"]
        }

        # 결과 확인 출력
        print("\n보고서용 태그:", report_data["report_tags"])
        print("\n태깅 근거:", report_data["tag_reasons"])
        print("\n상담사용 태그:", counsel_data["counsel_tags"])

        # 결과를 각각 저장하거나 프론트로 넘기기 쉽게 분리
        with open("보고서용_태그결과.json", "w", encoding="utf-8") as f:
            json.dump(report_data, f, ensure_ascii=False, indent=2)

        with open("상담사용_태그결과.json", "w", encoding="utf-8") as f:
            json.dump(counsel_data, f, ensure_ascii=False, indent=2)

else:
    print(f"❌ 파일을 찾을 수 없습니다: {file_path}")



보고서용 태그: ['정서불안', '학업스트레스', '부모의 기대/훈육 스트레스', '정서학대', '양육방임']

태깅 근거: 
- 정서불안: 아동이 불행하다고 느끼고, 성적과 엄마의 잔소리로 인해 마음이 아프다고 표현함.
- 학업스트레스: 성적 때문에 스트레스를 받고, 학원 다니는 것이 힘들다고 언급함.
- 부모의 기대/훈육 스트레스: 엄마가 성적에 대해 닥달하고, 성적이 떨어지면 비난하는 상황이 드러남.
- 정서학대: 엄마가 아동에게 바보 멍청이라고 하며 상처를 준 상황이 있음.
- 양육방임: 아동이 밤늦게까지 학원에 다니고, 집에서 돌봄을 받지 못하는 상황이 드러남.

상담사용 태그: ['가정폭력/양육방임', '부모•가족관계 문제', '정서•행동 문제', '학업•진로 스트레스']


<br>

In [ ]:
# 파일 경로 설정
file_path = "전체상담기록_2025-06-12_13-59.txt"

# 파일에서 대화 내용 읽고 태깅 실행
if os.path.exists(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation_data = file.read()
        result = tag_conversation(conversation_data)

        # 보고서 태그 + 사유
        report_data = {
            "report_tags": result["report_tags"],
            "tag_reasons": "\n".join(result["reasoning_text"].split('\n')[1:])  # 첫 줄 제거
        }

        # 상담사용 태그
        counsel_data = {
            "counsel_tags": result["counsel_tags"]
        }

        # 결과 확인 출력
        print("\n보고서용 태그:", report_data["report_tags"])
        print("\n태깅 근거:", report_data["tag_reasons"])
        print("\n상담사용 태그:", counsel_data["counsel_tags"])

        # 결과를 각각 저장하거나 프론트로 넘기기 쉽게 분리
        with open("보고서용_태그결과.json", "w", encoding="utf-8") as f:
            json.dump(report_data, f, ensure_ascii=False, indent=2)

        with open("상담사용_태그결과.json", "w", encoding="utf-8") as f:
            json.dump(counsel_data, f, ensure_ascii=False, indent=2)

else:
    print(f"❌ 파일을 찾을 수 없습니다: {file_path}")



보고서용 태그: ['신체학대', '가정학대', '양육방임', '정서불안', '형제자매 갈등']

태깅 근거: 
- 신체학대: 아동이 엄마에게 맞았다고 명확히 언급함.
- 가정학대: 엄마가 술을 마시고 폭력적으로 변해 아동을 때린다고 언급함.
- 양육방임: 엄마가 밤에 퇴근하고 형이 주로 아동을 돌본다고 언급함.
- 정서불안: 엄마의 폭력적인 행동으로 인해 무섭고 슬프다고 반복적으로 언급함.
- 형제자매 갈등: 형이 아동의 일기를 훔쳐보고 과자를 먹는 등 괴롭힌다고 언급함.

상담사용 태그: ['가정폭력/양육방임', '부모•가족관계 문제', '아동 신체학대', '정서•행동 문제']


#  

<br>

In [ ]:
# 파일 경로 설정
file_path = "전체상담기록_2025-07-03_02-44.txt"

# 파일에서 대화 내용 읽고 태깅 실행
if os.path.exists(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation_data = file.read()
        result = tag_conversation(conversation_data)

        # 보고서 태그 + 사유
        report_data = {
            "report_tags": result["report_tags"],
            "tag_reasons": "\n".join(result["reasoning_text"].split('\n')[1:])  # 첫 줄 제거
        }

        # 상담사용 태그
        counsel_data = {
            "counsel_tags": result["counsel_tags"]
        }

        # 결과 확인 출력
        print("\n보고서용 태그:", report_data["report_tags"])
        print("\n태깅 근거:", report_data["tag_reasons"])
        print("\n상담사용 태그:", counsel_data["counsel_tags"])

        # 결과를 각각 저장하거나 프론트로 넘기기 쉽게 분리
        with open("보고서용_태그결과.json", "w", encoding="utf-8") as f:
            json.dump(report_data, f, ensure_ascii=False, indent=2)

        with open("상담사용_태그결과.json", "w", encoding="utf-8") as f:
            json.dump(counsel_data, f, ensure_ascii=False, indent=2)

else:
    print(f"❌ 파일을 찾을 수 없습니다: {file_path}")



보고서용 태그: ['외모/신체 고민', '정서불안']

태깅 근거: 
- 외모/신체 고민: 아동이 자신의 외모에 대해 뚱뚱하다고 느끼고, 옷을 고를 때 고민이 있으며, 외모에 대한 걱정이 일상 생활에 영향을 미친다고 표현함.
- 정서불안: 아동이 외모에 대한 농담을 듣고 슬프고 불쾌하다고 느끼며, 자신이 못생겼다고 생각하여 행복하지 않다고 표현함.

상담사용 태그: ['정서•행동 문제']


<br>

In [ ]:
# 파일 경로 설정
file_path = "전체상담기록_2025-07-03_02-59.txt"

# 파일에서 대화 내용 읽고 태깅 실행
if os.path.exists(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        conversation_data = file.read()
        result = tag_conversation(conversation_data)

        # 보고서 태그 + 사유
        report_data = {
            "report_tags": result["report_tags"],
            "tag_reasons": "\n".join(result["reasoning_text"].split('\n')[1:])  # 첫 줄 제거
        }

        # 상담사용 태그
        counsel_data = {
            "counsel_tags": result["counsel_tags"]
        }

        # 결과 확인 출력
        print("\n보고서용 태그:", report_data["report_tags"])
        print("\n태깅 근거:", report_data["tag_reasons"])
        print("\n상담사용 태그:", counsel_data["counsel_tags"])

        # 결과를 각각 저장하거나 프론트로 넘기기 쉽게 분리
        with open("보고서용_태그결과.json", "w", encoding="utf-8") as f:
            json.dump(report_data, f, ensure_ascii=False, indent=2)

        with open("상담사용_태그결과.json", "w", encoding="utf-8") as f:
            json.dump(counsel_data, f, ensure_ascii=False, indent=2)

else:
    print(f"❌ 파일을 찾을 수 없습니다: {file_path}")



보고서용 태그: ['집단활동/교우관계 불안']

태깅 근거: 
- 집단활동/교우관계 불안: 아동이 친구가 없고, 친구를 사귀는 것에 대한 걱정과 불안을 표현하며, 친구들과 어울리는 것에 어려움을 느끼고 있음.

상담사용 태그: ['또래•사회관계 문제']
